In [4]:
# Imports
import pandas as pd
import plotly.express as px 

In [3]:
# read connection traffic data
df = pd.read_csv (r'/Users/ZOHELHAS/Desktop/connection-traffic-analysis/test.csv')
print (df)

    No.       Time           Source      Destination Protocol  Length  \
0     1   0.000000  192.168.137.195  192.168.137.195     ICMP     120   
1     2   0.000049  192.168.137.195  192.168.137.195     ICMP     120   
2     3   1.910611        127.0.0.1        127.0.0.1      DNS      92   
3     4   1.910696        127.0.0.1        127.0.0.1      DNS      92   
4     5   5.039988  192.168.137.195  192.168.137.195     ICMP     120   
5     6   5.040034  192.168.137.195  192.168.137.195     ICMP     120   
6     7   6.917354        127.0.0.1        127.0.0.1      DNS      81   
7     8   6.917440        127.0.0.1        127.0.0.1      DNS      81   
8     9  10.000054  192.168.137.195  192.168.137.195     ICMP     109   
9    10  10.000118  192.168.137.195  192.168.137.195     ICMP     109   
10   11  11.922655        127.0.0.1        127.0.0.1      DNS      81   
11   12  11.922733        127.0.0.1        127.0.0.1      DNS      81   
12   13  15.039992  192.168.137.195  192.168.137.19

In [7]:
fig = px.scatter(df, x='Time', y='Source', color='Protocol')
fig.show()

In [9]:
fig = px.box(df, x='Length', color='Source')
fig.show()

In [11]:
fig = px.histogram(df, x=df['Length'], marginal = "box",width=800, height=400)
fig.show()